In [1]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load churn dataset
dataset = openml.datasets.get_dataset(46915, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 3200
Validation set size: 800
Test set size: 1000


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
3444,1,120,415.0,0,1,27,179.6,142,30.53,262.8,103,22.34,239.9,81,10.80,11.1,2,3.00,1
2063,15,130,408.0,0,0,0,115.6,129,19.65,167.8,104,14.26,141.8,124,6.38,12.6,9,3.40,1
3714,26,66,510.0,0,0,0,201.3,95,34.22,152.8,66,12.99,233.2,101,10.49,7.5,4,2.03,1
2671,13,60,510.0,0,0,0,221.1,106,37.59,178.6,48,15.18,202.7,90,9.12,7.4,3,2.00,1
2154,35,134,408.0,0,0,0,202.7,105,34.46,224.9,90,19.12,253.9,108,11.43,12.1,7,3.27,0


In [3]:
X_train.dtypes

state                            category
account_length                      uint8
area_code                        category
international_plan               category
voice_mail_plan                  category
number_vmail_messages               uint8
total_day_minutes                 float64
total_day_calls                     uint8
total_day_charge                  float64
total_eve_minutes                 float64
total_eve_calls                     uint8
total_eve_charge                  float64
total_night_minutes               float64
total_night_calls                   uint8
total_night_charge                float64
total_intl_minutes                float64
total_intl_calls                    uint8
total_intl_charge                 float64
number_customer_service_calls       uint8
dtype: object

In [ ]:
from GRANDE import GRANDE

params = {
    'depth': 5,
    'n_estimators': 1024,

    'learning_rate_weights': 0.001,
    'learning_rate_index': 0.01,
    'learning_rate_values': 0.05,
    'learning_rate_leaf': 0.05,
    'learning_rate_embedding': 0.02,

    'use_category_embeddings': False,
    'embedding_dim_cat': 8,
    'use_numeric_embeddings': False,
    'embedding_dim_num': 8,
    'embedding_threshold': 1,
    'loo_cardinality': 10,


    'dropout': 0.2,
    'selected_variables': 0.8,
    'data_subset_fraction': 1.0,
    'bootstrap': False,
    'missing_values': False,

    'optimizer': 'adam', #nadam, radam, adamw, adam 
    'cosine_decay_restarts': False,
    'reduce_on_plateau_scheduler': True,
    'label_smoothing': 0.0,
    'use_class_weights': False,
    'focal_loss': False,
    'swa': False,
    'es_metric': True,  # if True use AUC for binary, MSE for regression, val_loss for multiclass


    'epochs': 250,
    'batch_size': 256,
    'early_stopping_epochs': 50,

    'use_freq_enc': False,
    'use_robust_scale_smoothing': False,
    'problem_type': 'binary',
    
    'random_seed': 42,
    'verbose': 2,
}

model_grande = GRANDE(params=params)

model_grande.fit(X=X_train,
          y=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict_proba(X_test)

self.params {'depth': 5, 'n_estimators': 1024, 'learning_rate_weights': 0.001, 'learning_rate_index': 0.01, 'learning_rate_values': 0.05, 'learning_rate_leaf': 0.05, 'learning_rate_embedding': 0.02, 'use_category_embeddings': False, 'embedding_dim_cat': 8, 'use_numeric_embeddings': False, 'embedding_dim_num': 8, 'embedding_threshold': 1, 'loo_cardinality': 10, 'dropout': 0.2, 'selected_variables': 0.8, 'data_subset_fraction': 1.0, 'bootstrap': False, 'missing_values': False, 'optimizer': 'adam', 'cosine_decay_restarts': False, 'reduce_on_plateau_scheduler': True, 'label_smoothing': 0.0, 'use_class_weights': False, 'focal_loss': False, 'swa': False, 'es_metric': True, 'epochs': 250, 'batch_size': 256, 'early_stopping_epochs': 50, 'use_freq_enc': False, 'use_robust_scale_smoothing': False, 'problem_type': 'binary', 'random_seed': 42, 'verbose': 2, 'device': 'cuda:0', 'objective': 'binary'}
{'self.column_names_dataframe': ['state',
                                 'account_length',
      

/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(


Epoch 001 | TrainLoss: 0.5474 | ValLoss: 0.4453 | ValAcc: 0.8462 | ValAUC: 0.7365 | ValF1: 0.4584 | Time: 7.30s
Epoch 002 | TrainLoss: 0.3948 | ValLoss: 0.3909 | ValAcc: 0.8462 | ValAUC: 0.7867 | ValF1: 0.4584 | Time: 0.07s
Epoch 003 | TrainLoss: 0.3378 | ValLoss: 0.3544 | ValAcc: 0.8512 | ValAUC: 0.8047 | ValF1: 0.4911 | Time: 0.07s
Epoch 004 | TrainLoss: 0.2971 | ValLoss: 0.3175 | ValAcc: 0.8688 | ValAUC: 0.8550 | ValF1: 0.6018 | Time: 0.07s
Epoch 005 | TrainLoss: 0.2589 | ValLoss: 0.2886 | ValAcc: 0.8712 | ValAUC: 0.8808 | ValF1: 0.6189 | Time: 0.07s
Epoch 006 | TrainLoss: 0.2375 | ValLoss: 0.2643 | ValAcc: 0.8775 | ValAUC: 0.8945 | ValF1: 0.6479 | Time: 0.07s
Epoch 007 | TrainLoss: 0.2188 | ValLoss: 0.2462 | ValAcc: 0.8912 | ValAUC: 0.9017 | ValF1: 0.7092 | Time: 0.07s
Epoch 008 | TrainLoss: 0.2062 | ValLoss: 0.2279 | ValAcc: 0.9125 | ValAUC: 0.9067 | ValF1: 0.7929 | Time: 0.07s
Epoch 009 | TrainLoss: 0.1965 | ValLoss: 0.2168 | ValAcc: 0.9200 | ValAUC: 0.9133 | ValF1: 0.8123 | Time

/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(


In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)

In [7]:
binary_indices = []
low_cardinality_indices = []
high_cardinality_indices = []
num_columns = []
for column_index, column in enumerate(X_train.columns):
    if column_index in categorical_feature_indices or X_train.iloc[:,column_index].dtype.name == 'category' or X_train.iloc[:,column_index].dtype.name == 'object':
        if len(X_train.iloc[:,column_index].unique()) <= 2:
            binary_indices.append(column)
        if len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(column)
        else:
            high_cardinality_indices.append(column)
    else:
        num_columns.append(column)     
cat_columns = [col for col in X_train.columns if col not in num_columns]
categorical_feature_indices = [X_train.columns.get_loc(col) for col in cat_columns]


In [8]:
if len(num_columns) > 0:
    mean_train_num = X_train[num_columns].mean(axis=0).iloc[0]
    X_train[num_columns] = X_train[num_columns].fillna(mean_train_num)
    X_valid[num_columns] = X_valid[num_columns].fillna(mean_train_num)
    X_test[num_columns] = X_test[num_columns].fillna(mean_train_num)
if len(cat_columns) > 0:
    mode_train_cat = X_train[cat_columns].mode(axis=0).iloc[0]
    X_train[cat_columns] = X_train[cat_columns].fillna(mode_train_cat)
    X_valid[cat_columns] = X_valid[cat_columns].fillna(mode_train_cat)
    X_test[cat_columns] = X_test[cat_columns].fillna(mode_train_cat)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [9]:
encoder_ordinal = ce.OrdinalEncoder(cols=binary_indices)
encoder_ordinal.fit(X_train)
X_train = encoder_ordinal.transform(X_train)
X_valid = encoder_ordinal.transform(X_valid)  
X_test = encoder_ordinal.transform(X_test)       

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
if params['problem_type'] == 'regression':
    from xgboost import XGBRegressor
    model_xgb = XGBRegressor(n_estimators=1000, early_stopping_rounds=20)
    model_xgb.fit(X_train, 
                    y_train, 
                    eval_set=[(X_valid, y_valid)], 
                )
    preds_xgb = model_xgb.predict(X_test)
else:
    from xgboost import XGBClassifier
    model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
    model_xgb.fit(X_train, 
                    y_train, 
                    #sample_weight=calculate_sample_weights(y_train), 
                    eval_set=[(X_valid, y_valid)], 
                    #sample_weight_eval_set=[calculate_sample_weights(y_valid)]
                )


    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-logloss:0.28315
[1]	validation_0-logloss:0.23763
[2]	validation_0-logloss:0.20618
[3]	validation_0-logloss:0.18811
[4]	validation_0-logloss:0.17719
[5]	validation_0-logloss:0.16986
[6]	validation_0-logloss:0.16414
[7]	validation_0-logloss:0.16136
[8]	validation_0-logloss:0.15800
[9]	validation_0-logloss:0.15785
[10]	validation_0-logloss:0.15713
[11]	validation_0-logloss:0.15692
[12]	validation_0-logloss:0.15799
[13]	validation_0-logloss:0.15559
[14]	validation_0-logloss:0.15664
[15]	validation_0-logloss:0.15613
[16]	validation_0-logloss:0.15517
[17]	validation_0-logloss:0.15363
[18]	validation_0-logloss:0.15438
[19]	validation_0-logloss:0.15495
[20]	validation_0-logloss:0.15732
[21]	validation_0-logloss:0.15925
[22]	validation_0-logloss:0.15958
[23]	validation_0-logloss:0.15969
[24]	validation_0-logloss:0.16180
[25]	validation_0-logloss:0.16142
[26]	validation_0-logloss:0.16153
[27]	validation_0-logloss:0.16176
[28]	validation_0-logloss:0.16241
[29]	validation_0-loglos

In [11]:
if params['problem_type'] == 'regression':
    from catboost import CatBoostRegressor, Pool

    model_catboost = CatBoostRegressor(n_estimators=1000, 
                                        early_stopping_rounds=20)
    train_data = Pool(
            data=X_train_raw,
            label=y_train,
            cat_features=categorical_feature_indices,
        )

    eval_data = Pool(
            data=X_valid_raw,
            label=y_valid,
            cat_features=categorical_feature_indices,
        )

    model_catboost.fit(X=train_data, 
                    eval_set=eval_data)




    preds_catboost = model_catboost.predict(X_test_raw)
else:
    from catboost import CatBoostClassifier, Pool

    model_catboost = CatBoostClassifier(n_estimators=1000, 
                                        early_stopping_rounds=20)
    train_data = Pool(
            data=X_train_raw,
            label=y_train,
            cat_features=categorical_feature_indices,
            #weight=calculate_sample_weights(y_train)
        )

    eval_data = Pool(
            data=X_valid_raw,
            label=y_valid,
            cat_features=categorical_feature_indices,
            #weight=calculate_sample_weights(y_valid),
        )

    model_catboost.fit(X=train_data, 
                    eval_set=eval_data)



    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.042236
0:	learn: 0.6403699	test: 0.6408011	best: 0.6408011 (0)	total: 53.5ms	remaining: 53.4s
1:	learn: 0.6121572	test: 0.6148520	best: 0.6148520 (1)	total: 56.1ms	remaining: 28s
2:	learn: 0.5688593	test: 0.5737767	best: 0.5737767 (2)	total: 62ms	remaining: 20.6s
3:	learn: 0.5353572	test: 0.5418521	best: 0.5418521 (3)	total: 65.5ms	remaining: 16.3s
4:	learn: 0.5080383	test: 0.5151234	best: 0.5151234 (4)	total: 69.9ms	remaining: 13.9s
5:	learn: 0.4885596	test: 0.4971972	best: 0.4971972 (5)	total: 74.7ms	remaining: 12.4s
6:	learn: 0.4654316	test: 0.4759393	best: 0.4759393 (6)	total: 79.2ms	remaining: 11.2s
7:	learn: 0.4445447	test: 0.4561330	best: 0.4561330 (7)	total: 82.5ms	remaining: 10.2s
8:	learn: 0.4288701	test: 0.4418229	best: 0.4418229 (8)	total: 85.3ms	remaining: 9.39s
9:	learn: 0.4052309	test: 0.4181304	best: 0.4181304 (9)	total: 88.3ms	remaining: 8.74s
10:	learn: 0.3831187	test: 0.3959788	best: 0.3959788 (10)	total: 94.1ms	remaining: 8.46s
11:	learn: 0.37

In [12]:
if params['problem_type'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif params['problem_type'] == 'multiclass':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_grande)
    root_mean_squared_error = np.sqrt(((y_test - preds_grande) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_grande)

    print('MAE GRANDE:', mean_absolute_error)
    print('RMSE GRANDE:', root_mean_squared_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_xgb)
    root_mean_squared_error = np.sqrt(((y_test - preds_xgb) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_xgb)

    print('MAE XGB:', mean_absolute_error)
    print('RMSE XGB:', root_mean_squared_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_catboost)
    root_mean_squared_error = np.sqrt(((y_test - preds_catboost) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_catboost)

    print('MAE CatBoost:', mean_absolute_error)
    print('RMSE CatBoost:', root_mean_squared_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

Accuracy GRANDE: 0.954
F1 Score GRANDE: 0.8976494984825425
ROC AUC GRANDE: 0.922450889462646


Accuracy XGB: 0.952
F1 Score XGB: 0.8902857142857143
ROC AUC XGB: 0.9152775340703048


Accuracy CatBoost: 0.957
F1 Score CatBoost: 0.8999941857084715
ROC AUC CatBoost: 0.9137693329656831


